In [ ]:
## Chapter04から使用するテキストデータをコピーする（事前にChapter04/Sentiment.ipynbを実行してください）
!cp -r ../Chapter04/sisyou_db ./

In [1]:
import os
import time

import pandas as pd
import requests
import spacy

## モデルのロード
nlp = spacy.load("ja_ginza_electra")
data_dir = "sisyou_db"

error We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like megagonlabs/transformers-ud-japanese-electra-base-ginza-510 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.
error Can't load the configuration of '/var/folders/n8/f5dk0msn63n2039s3mfg66wc0000gn/T/tmpxjq1k98_/config.json'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/var/folders/n8/f5dk0msn63n2039s3mfg66wc0000gn/T/tmpxjq1k98_/config.json' is the correct path to a directory containing a config.json file
trying to download model from huggingface hub: megagonlabs/transformers-ud-japanese-electra-base-ginza-510 ...


succeded


In [2]:
# 1.労災データベースからのファイルダウンロード
## 労働災害データベースからのダウンロードを下記で行っていますが、
## 本NotebookでははChapter04からコピーしているため、ダウンロード自体は行われません。
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

years = [28, 29]
months = list(range(1, 13))

for y in years:
    for m in months:
        file_name = f"{data_dir}/sisyou_db_h{y}_{m:02d}.xlsx"
        print(file_name)
        if not os.path.exists(file_name):
            file_url = f"https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_h{y}_{m:02d}.xlsx"
            res = requests.get(file_url)
            with open(file_name, 'wb') as f:
                f.write(res.content)
            time.sleep(2)    

sisyou_db/sisyou_db_h28_01.xlsx
sisyou_db/sisyou_db_h28_02.xlsx
sisyou_db/sisyou_db_h28_03.xlsx
sisyou_db/sisyou_db_h28_04.xlsx
sisyou_db/sisyou_db_h28_05.xlsx
sisyou_db/sisyou_db_h28_06.xlsx
sisyou_db/sisyou_db_h28_07.xlsx
sisyou_db/sisyou_db_h28_08.xlsx
sisyou_db/sisyou_db_h28_09.xlsx
sisyou_db/sisyou_db_h28_10.xlsx
sisyou_db/sisyou_db_h28_11.xlsx
sisyou_db/sisyou_db_h28_12.xlsx
sisyou_db/sisyou_db_h29_01.xlsx
sisyou_db/sisyou_db_h29_02.xlsx
sisyou_db/sisyou_db_h29_03.xlsx
sisyou_db/sisyou_db_h29_04.xlsx
sisyou_db/sisyou_db_h29_05.xlsx
sisyou_db/sisyou_db_h29_06.xlsx
sisyou_db/sisyou_db_h29_07.xlsx
sisyou_db/sisyou_db_h29_08.xlsx
sisyou_db/sisyou_db_h29_09.xlsx
sisyou_db/sisyou_db_h29_10.xlsx
sisyou_db/sisyou_db_h29_11.xlsx
sisyou_db/sisyou_db_h29_12.xlsx


In [3]:
# 2. 分類対象データの読み込み
## 労災データの読み込み
def read_rousai_db(data_dir, year_months):
    dfs = []
    for y, m in year_months:
        file_name = f"{data_dir}/sisyou_db_h{y}_{m:02d}.xlsx"
        df = pd.read_excel(file_name, skiprows=[1])
        dfs.append(df)

    df_ret = pd.concat(dfs)
    df_ret = df_ret.rename(columns={"事故の型": "事故の型_コード",
                                    "Unnamed: 20": "事故の型_名前"})
    return df_ret

## 訓練データ・テストデータの読み込み
year_months_train = [(28, m) for m in months]
year_months_dev = [(29, 1)]
year_months_test = [(29, 2)]
df_train = read_rousai_db(data_dir, year_months_train)
df_dev = read_rousai_db(data_dir, year_months_dev)
df_test = read_rousai_db(data_dir, year_months_test)

In [4]:
# 3.分類クラスを頻度上位10クラス+それ以外の11クラスに絞り込み
names = df_train["事故の型_名前"].unique()
top10_category = df_train["事故の型_名前"].value_counts()[0:10].index.to_list()
name2label = {name: top10_category.index(name) if name in top10_category else len(top10_category) for i, name in enumerate(names)}
print(top10_category)


['転倒', '墜落、転落', '動作の反動、無理な動作', 'はさまれ、巻き込まれ', '切れ、こすれ', '交通事故（道路）', '飛来、落下', '激突', '激突され', '高温・低温の物との接触']


In [5]:
# 4.spaCy Command Line Interface 用の訓練データ・テストデータのファイルを生成
from spacy.tokens import DocBin

def convert_docs(df_target, dic_label):
    list_docs = []
    for index, entry in df_target.iterrows():
        doc = nlp.make_doc(entry["災害状況"])
        for label,val in dic_label.items():
            if(label == entry["事故の型_名前"]):
                doc.cats[str(val)] = 1
            else:
                doc.cats[str(val)] = 0

        list_docs.append(doc)
    return list_docs

docs_train = convert_docs(df_train, name2label)
doc_bin_train = DocBin(docs=docs_train)
doc_bin_train.to_disk("./train_ja.spacy")

docs_dev = convert_docs(df_dev, name2label)
doc_bin_dev = DocBin(docs=docs_dev)
doc_bin_dev.to_disk("./dev_ja.spacy")

In [6]:
# 5.分類器の学習を実行 => 文書分類モデルを./rousai_classifier　に保存
## configファイルの生成
!python -m spacy init fill-config ./base_ginza_textcat.cfg ./ginza_textcat.cfg
## 訓練の実行
!python -m spacy train ./ginza_textcat.cfg --output ./rousai_classifier

✔ Auto-filled config with all values
✔ Saved config
ginza_textcat.cfg
You can now add your data and train your pipeline:
python -m spacy train ginza_textcat.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
✔ Created output directory: rousai_classifier
ℹ Saving to output directory: rousai_classifier
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-19 22:31:05,736] [INFO] Set up nlp object from config
[2023-04-19 22:31:05,741] [INFO] Pipeline: ['textcat']
[2023-04-19 22:31:05,743] [INFO] Created vocabulary
[2023-04-19 22:31:05,744] [INFO] Finished initializing nlp object
[2023-04-19 22:32:03,505] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.0
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.08        0.00    0.0

In [7]:
# 6.性能がベストだった文書分類モデルを読み込み、評価用データに適用
nlp_best = spacy.load("rousai_classifier/model-best")
counter = 0
prec = 0
for index, entry in df_test.iterrows():
    doc = nlp_best(entry["災害状況"])
    label = entry["事故の型_名前"]

    ## 1番スコアの高いクラスを予測値とする
    for k,v in sorted(doc.cats.items(),key=lambda x:x[1],reverse=True):
        toprank = int(k)
        break
    ## 評価用データのラベルを分類クラスIDに変換
    answer = name2label[label]

    if toprank == answer:
        prec += 1
    counter += 1

print("{}({}/{})".format(prec/counter,prec,counter))

0.7373857767183154(1856/2517)
